In [ ]:
import os
import numpy as np
import pandas as pd

from pointtree.instance_segmentation import MultiStageAlgorithm
from pointtree.evaluation import get_detections, get_instance_detection_metrics, get_instance_segmentation_metrics, get_semantic_segmentation_metrics

In [ ]:
datasets = {
    'TreeML': {
        '2023-01-09_5_1_37': {
            'file_path': '<insert path>/Data/TreeML/2_semantic_segmentation/2023-01-09_5_1_37.csv',
            'street': '2023-01-09\_5\_1\_37',
            'part': '',
        },
        '2023-01-09_17_2_18': {
            'file_path': '<insert path>/Data/TreeML/2_semantic_segmentation/2023-01-09_17_2_18.csv',
            'street': '2023-01-09\_17\_2\_18',
            'part': ''
        },
        '2023-01-10_7_6': {
            'file_path': '<insert path>/Data/TreeML/2_semantic_segmentation/2023-01-10_7_6.csv',
            'street': '2023-01-10\_7\_6',
            'part': ''
        },
        '2023-01-12_35_34': {
            'file_path': '<insert path>/Data/TreeML/2_semantic_segmentation/2023-01-12_35_34.csv',
            'street': '2023-01-12\_35\_34',
            'part': ''
        },
        '2023-01-12_65_64': {
            'file_path': '<insert path>/Data/TreeML/2_semantic_segmentation/2023-01-12_65_64.csv',
            'street': '2023-01-12\_65\_64',
            'part': ''
        },
        '2023-01-16_44': {
            'file_path': '<insert path>/Data/TreeML/2_semantic_segmentation/2023-01-16_44.csv',
            'street': '2023-01-16\_44',
            'part': ''
        }
    },
    'Essen': {
        'altendorfer_part_1': {
            'file_path': '<insert path>/Data/Essen/2_semantic_segmentation/Altendorfer_p1_min_1.csv',
            'street': 'Altendorfer Straße',
            'part': 'part 1'
        },
        'altendorfer_part_2': {
            'file_path': '<insert path>/Data/Essen/2_semantic_segmentation/Altendorfer_p2_min_1.csv',
            'street': 'Altendorfer Straße',
            'part': 'part 2'
        },
        'altenessener_part_4': {
            'file_path': '<insert path>/Data/Essen/2_semantic_segmentation/Essen3_p2_min_1.csv',
            'street': 'Altenessener Straße',
            'part': 'part 4'
        },
        'altenessener_part_5': {
            'file_path': '<insert path>/Data/Essen/2_semantic_segmentation/Essen3_p3_min_1.csv',
            'street': 'Altenessener Straße',
            'part': 'part 5'
        }
    },
    'Hamburg': {
        'armgart_straße_part_1': {
            'file_path': '<insert path>/Data/Hamburg/2_semantic_segmentation/000274_v2_min_1.csv',
            'street': 'Armgartstraße',
            'part': 'part 1'

        },
        'armgart_straße_part_2': {
            'file_path': '<insert path>/Data/Hamburg/2_semantic_segmentation/000275_000276_min_1.csv',
            'street': 'Armgartstraße',
            'part': 'part 2'
        }
    }
}

output_dir = '<insert path>/Data/3DGeoInfoExperiments/Watershed-Basic'
min_iou_match = 0.3

In [ ]:
os.makedirs(output_dir, exist_ok=True)

for dataset in datasets:
    os.makedirs(os.path.join(output_dir, dataset), exist_ok=True)
    for file_id, file_infos in datasets[dataset].items():
        for (experiment_label, semclass_id_column, specificclass_id_column) in [
            ('gt', 'semclassid', 'specificclassid'),
            ('dl', 'semclassidpredicted', 'specificclassidpredicted')
        ]:
            if experiment_label == "segmentation_ground_truth" and dataset == "TreeML":
                continue
            point_cloud = pd.read_csv(file_infos['file_path'])

            point_cloud = point_cloud.rename({'//X': 'X'}, axis=1)
            point_cloud = point_cloud.rename(str.lower, axis=1)

            columns_to_keep = ['x', 'y', 'z', 'incomplete', 'low_density', 'instance_id', 'semclassidpredicted', 'specificclassidpredicted']

            if 'classification' in point_cloud.columns:
                columns_to_keep.append('classification')
            if 'semclassid' in point_cloud.columns:
                columns_to_keep.append('semclassid')
            if 'specificclassid' in point_cloud.columns:
                columns_to_keep.append('specificclassid')

            point_cloud = point_cloud[columns_to_keep]

            class_mapping = {
                (0, 0): 0,
                (1, 0): 0,
                (1, 1): 1,
                (1, 2): 2,
                (1, 3): 3,
                (2, 0): 4,
                (3, 0): 4,
                (4, 0): 4,
                (4, 1): 4
            }
                
            print("Process", file_id, experiment_label)

            instance_seg_folder = os.path.join(output_dir, 'Data', dataset, '3_instance_segmentation')
            os.makedirs(instance_seg_folder, exist_ok=True)
            metrics_folder = os.path.join(output_dir, 'Metrics', dataset, file_id)
            os.makedirs(metrics_folder, exist_ok=True)
            visualization_folder = os.path.join(output_dir, 'Data', dataset, 'visualizations', file_id, experiment_label)
            os.makedirs(visualization_folder, exist_ok=True)

            if experiment_label == "dl":
                if dataset != "TreeML":
                    class_map = {
                        'other': 0,
                        'trunk': 1,
                        'crown': 2,
                        'branch': 3,
                        'low_vegetation': 4
                    }
                    point_cloud["classification_target"] = np.vectorize(lambda x, y: class_mapping[(x, y)])(
                        point_cloud["semclassid"].to_numpy(), point_cloud["specificclassid"].to_numpy()
                    )
                    point_cloud["classification_prediction"] = np.vectorize(lambda x, y: class_mapping[(x, y)])(
                        point_cloud["semclassidpredicted"].to_numpy(), point_cloud["specificclassidpredicted"].to_numpy()
                    )

                else:
                    class_map = {
                        'other': 0,
                        'building': 1,
                        'tree': 2
                    }
                    target_class_mapping = {
                        0: 0,
                        1: 0,
                        2: 1,
                    }
                    prediction_class_mapping = {
                        (0, 0): 0,
                        (1, 0): 0,
                        (1, 1): 1,
                        (1, 2): 1,
                        (1, 3): 1,
                        (2, 0): 0
                    }
                    point_cloud["classification_target"] = np.vectorize(lambda x: target_class_mapping[x])(
                        point_cloud["classification"].to_numpy()
                    )
                    point_cloud["classification_prediction"] = np.vectorize(lambda x, y: prediction_class_mapping[(x, y)])(
                        point_cloud["semclassidpredicted"].to_numpy(), point_cloud["specificclassidpredicted"].to_numpy()
                    )
                semantic_segmentation_metrics = get_semantic_segmentation_metrics(point_cloud['classification_target'].to_numpy(),
                                                                                point_cloud["classification_prediction"].to_numpy(),
                                                                                class_map)
                semantic_segmentation_metrics["Dataset"] = dataset
                semantic_segmentation_metrics["FileID"] = file_id
                semantic_segmentation_metrics["Street"] = file_infos['street']
                semantic_segmentation_metrics["Part"] = file_infos['part']
                semantic_segmentation_metrics.to_csv(os.path.join(metrics_folder, f'{file_id}_semantic_segmentation.csv'), index=False)

            point_cloud["classification"] = np.vectorize(lambda x, y: class_mapping[(x, y)])(
                point_cloud[semclass_id_column].to_numpy(), point_cloud[specificclass_id_column].to_numpy()
            )

            point_cloud = point_cloud.drop(["classification_target", "classification_prediction", "semclassid", "specificclassid", "semclassidpredicted", "specificclassidpredicted"],
                                axis=1, errors='ignore')

            segmentation_algorithm = MultiStageSegmentationAlgorithm(trunk_class_id=1, crown_class_id=2,
                                                                     branch_class_id=3,
                                                                     algorithm="watershed_tree_top_positions",
                                                                     correct_watershed=False,
                                                                     grid_size=0.03,
                                                                     visualization_folder=visualization_folder)
            point_cloud["instance_id_predicted"] = segmentation_algorithm.segment(point_cloud,
                                                                                    f'{dataset}_{file_id}_{experiment_label}')
            
            point_cloud.to_csv(os.path.join(instance_seg_folder, f'{file_id}_{experiment_label}.csv'), index=False)

            runtime_stats = segmentation_algorithm.runtime_stats()
            runtime_stats["Dataset"] = dataset
            runtime_stats["FileID"] = file_id
            runtime_stats["Street"] = file_infos['street']
            runtime_stats["Part"] = file_infos['part']
            runtime_stats["SemanticSegmentation"] = experiment_label
            runtime_stats.to_csv(os.path.join(metrics_folder, f'{file_id}_{experiment_label}_runtime.csv'), index=False)

            matched_gts, matched_preds, iou_matrix = get_detections(
                point_cloud["instance_id"].to_numpy(),
                point_cloud["instance_id_predicted"].to_numpy(),
                min_iou_match=min_iou_match
            )

            instance_detection_metrics = get_instance_detection_metrics(point_cloud["instance_id"].to_numpy(), 
                                                                    point_cloud["instance_id_predicted"].to_numpy(),
                                                                    matched_gts,
                                                                    matched_preds)
            instance_segmentation_metrics = get_instance_segmentation_metrics(point_cloud["instance_id"].to_numpy(), 
                                                                    point_cloud["instance_id_predicted"].to_numpy(),
                                                                    matched_gts,
                                                                    matched_preds)

            instance_detection_metrics["Dataset"] = dataset
            instance_detection_metrics["FileID"] = file_id
            instance_detection_metrics["Street"] = file_infos['street']
            instance_detection_metrics["Part"] = file_infos['part']
            instance_detection_metrics["SemanticSegmentation"] = experiment_label
            instance_detection_metrics = instance_detection_metrics[['Dataset', 'FileID', 'Street', 'Part', 'SemanticSegmentation', 'TP', 'FP', 'FN', 'Precision', 'Recall']]
            instance_detection_metrics.to_csv(os.path.join(metrics_folder, f'{file_id}_{experiment_label}_instance_detection.csv'),
                                              index=False)

            # instance_segmentation_metrics["mIoU"] = instance_segmentation_metrics["iou"].mean() if len(instance_segmentation_metrics) > 0 else np.nan
            # instance_segmentation_metrics["mPrecision"] = instance_segmentation_metrics["precision"].mean() if len(instance_segmentation_metrics) > 0 else np.nan
            # instance_segmentation_metrics["mRecall"] = instance_segmentation_metrics["recall"].mean() if len(instance_segmentation_metrics) > 0 else np.nan
            instance_segmentation_metrics["Dataset"] = dataset
            instance_segmentation_metrics["FileID"] = file_id
            instance_segmentation_metrics["Street"] = file_infos['street']
            instance_segmentation_metrics["Part"] = file_infos['part']
            instance_segmentation_metrics["SemanticSegmentation"] = experiment_label
            instance_segmentation_metrics = instance_segmentation_metrics[['Dataset', 'FileID', 'Street', 'Part', 'SemanticSegmentation', 'IdTarget', 'IdPred', 'IoU', 'Precision', 'Recall']]
            instance_segmentation_metrics.to_csv(os.path.join(metrics_folder, f'{file_id}_{experiment_label}_instance_segmentation.csv'),
                                                 index=False)


In [ ]:
for dataset in datasets:
    metrics_folder = os.path.join(output_dir, 'Metrics', dataset)

    for (experiment_label, semclass_id_column, specificclass_id_column) in [
            ('gt', 'semclassid', 'specificclassid'),
            ('dl', 'semclassidpredicted', 'specificclassidpredicted')
        ]:
        runtime_results = []
        semantic_segmentation_results = []
        detection_results = []
        segmentation_results = []
        for file_id, file_path in datasets[dataset].items():
                print("file_id", file_id)
                runtime_metrics_file = os.path.join(metrics_folder, file_id,
                                                                f'{file_id}_{experiment_label}_runtime.csv')
                if os.path.exists(runtime_metrics_file):
                    runtime_results.append(pd.read_csv(runtime_metrics_file))
                if experiment_label == "dl":
                    semantic_segmentation_metrics_file = os.path.join(metrics_folder, file_id,
                                                                    f'{file_id}_semantic_segmentation.csv')
                    if os.path.exists(semantic_segmentation_metrics_file):
                        semantic_segmentation_results.append(pd.read_csv(semantic_segmentation_metrics_file))

                detection_metrics_file = os.path.join(metrics_folder, file_id, f'{file_id}_{experiment_label}_instance_detection.csv')
                if os.path.exists(detection_metrics_file):
                    detection_results.append(pd.read_csv(detection_metrics_file))

                segmentation_metrics_file = os.path.join(metrics_folder, file_id,
                                                                f'{file_id}_{experiment_label}_instance_segmentation.csv')

                if os.path.exists(segmentation_metrics_file):
                    segmentation_results.append(pd.read_csv(segmentation_metrics_file))

        if len(runtime_results) > 0:
            runtime_results = pd.concat(runtime_results)
            runtime_results.to_csv(os.path.join(metrics_folder, f'runtime_{dataset}.csv'), index=False)

        if experiment_label == "dl" and len(semantic_segmentation_results) > 0:
            semantic_segmentation_results = pd.concat(semantic_segmentation_results)
            semantic_segmentation_results.to_csv(os.path.join(metrics_folder, f'semantic_segmentation_metrics_{dataset}.csv'), index=False)

        if len(segmentation_results) > 0:
            segmentation_results = pd.concat(segmentation_results)
            segmentation_results_aggregated = segmentation_results[['Dataset', 'FileID', 'Street', 'Part', 'SemanticSegmentation', 'IoU', 'Precision', 'Recall']].groupby(['Dataset', 'FileID', 'Street', 'Part', 'SemanticSegmentation']).mean()
            segmentation_results_aggregated[segmentation_results_aggregated.index.names] = segmentation_results_aggregated.index.to_list()
            segmentation_results_aggregated.index = np.arange(len(segmentation_results_aggregated), dtype=np.int64)
            segmentation_results_aggregated = pd.DataFrame(segmentation_results_aggregated)
            segmentation_results_aggregated = segmentation_results_aggregated.rename({'IoU': 'mIoU', 'Precision': 'mPrecision', 'Recall': 'mRecall'}, axis=1)
            segmentation_results_total = {
                "Dataset": dataset,
                "FileID": "total",
                "Street": "Total",
                "Part": "",
                "SemanticSegmentation": experiment_label,
                "mIoU": segmentation_results["IoU"].mean() if len(segmentation_results) > 0 else np.nan,
                "mPrecision": segmentation_results["Precision"].mean() if len(segmentation_results) > 0 else np.nan,
                "mRecall": segmentation_results["Recall"].mean() if len(segmentation_results) > 0 else np.nan
            }
            segmentation_results = pd.concat([segmentation_results_aggregated, pd.DataFrame([segmentation_results_total])])
            segmentation_results = segmentation_results[['Dataset', 'FileID', 'Street', 'Part', 'SemanticSegmentation', 'mIoU', 'mPrecision', 'mRecall']]
            segmentation_results.to_csv(os.path.join(metrics_folder, f'instance_segmentation_metrics_{dataset}_{experiment_label}.csv'), index=False)

        if len(detection_results) > 0:
            detection_results = pd.concat(detection_results)
            detection_results_total = {
                "Dataset": dataset,
                "FileID": "total",
                "Street": "Total",
                "Part": "",
                "SemanticSegmentation": experiment_label,
                "TP": detection_results["TP"].sum(),
                "FP": detection_results["FP"].sum(),
                "FN": detection_results["FN"].sum(),
                "Precision": detection_results["TP"].sum() / (detection_results["TP"].sum() + detection_results["FP"].sum()),
                "Recall": detection_results["TP"].sum() / (detection_results["TP"].sum() + detection_results["FN"].sum()),
                # "mIoU": segmentation_results["iou"].mean() if len(segmentation_results) > 0 else np.nan,
                # "mPrecision": segmentation_results["precision"].mean() if len(segmentation_results) > 0 else np.nan,
                # "mRecall": segmentation_results["recall"].mean() if len(segmentation_results) > 0 else np.nan
            }
            detection_results = pd.concat([detection_results, pd.DataFrame([detection_results_total])])
            detection_results['Precision'] = np.round(detection_results['Precision'].to_numpy(), 2)
            detection_results['Recall'] = np.round(detection_results['Recall'].to_numpy(), 2)
            detection_results = detection_results[['Dataset', 'FileID', 'Street', 'Part', 'SemanticSegmentation', 'TP', 'FP', 'FN', 'Precision', 'Recall']]
            detection_results.to_csv(os.path.join(metrics_folder, f'instance_detection_metrics_{dataset}_{experiment_label}.csv'), index=False)



In [ ]:
segmentation_results_aggregated[segmentation_results_aggregated.index.names] = segmentation_results_aggregated.index.to_list()